In [1]:
### Encoding categorical variables
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np


ca_df = pd.read_csv("marketing_customer_analysis_clean.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'marketing_customer_analysis_clean.csv'

In [ ]:
ca_df.head()

In [ ]:
ca_df = ca_df.drop(columns = ["unnamed:_0", "customer", "effective_to_date"])

In [ ]:
ca_df

In [ ]:
categoricals = ca_df.select_dtypes(include=object)
numericals = ca_df.select_dtypes(include = np.number)

In [ ]:
ca_df["education"].unique()

In [ ]:
ordinals = categoricals[["coverage","education","vehicle_size"]]
nominals = categoricals.drop(columns=["coverage","education","vehicle_size"])

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
enc = OrdinalEncoder(categories=[["Basic","Extended","Premium"]])

ordinals["coverage"] = enc.fit_transform(ordinals[["coverage"]])

enc = OrdinalEncoder(categories=[["High School or Below", "College", "Bachelor","Master", "Doctor"]])

ordinals["education"] = enc.fit_transform(ordinals[["education"]])

enc = OrdinalEncoder(categories=[["Small","Medsize","Large"]])

ordinals["vehicle_size"] = enc.fit_transform(ordinals[["vehicle_size"]])

In [ ]:
ordinals

In [ ]:
nominals = pd.get_dummies(nominals)

In [ ]:
nominals

### Numericals

In [ ]:
numericals = ca_df.select_dtypes(include='number')
numericals.columns

In [ ]:
#dropping nas
numericals = numericals.dropna()

In [ ]:
numericals.columns

In [ ]:
#dropping duplicates
numericals = numericals.drop_duplicates()

In [ ]:
numericals.columns

### CONCAT

In [ ]:
final_df = pd.concat([numericals,ordinals,nominals], axis=1)

In [ ]:
final_df.columns

### dropping nans

In [ ]:
final_df.dropna(inplace=True)

### Lab instructions : 

1) Compute R2 for the first model iteration + feature importance plot
2) Run the 2nd iteration of the model, adding the categorical data

### X-y Split

In [ ]:
X = final_df[['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies',
       'month', 'coverage', 'education', 'vehicle_size', 'state_Arizona',
       'state_California', 'state_Nevada', 'state_Oregon', 'state_Washington',
       'response_No', 'response_Yes', 'employmentstatus_Disabled',
       'employmentstatus_Employed', 'employmentstatus_Medical Leave',
       'employmentstatus_Retired', 'employmentstatus_Unemployed', 'gender_F',
       'gender_M', 'location_code_Rural', 'location_code_Suburban',
       'location_code_Urban', 'marital_status_Divorced',
       'marital_status_Married', 'marital_status_Single',
       'policy_type_Corporate Auto', 'policy_type_Personal Auto',
       'policy_type_Special Auto', 'policy_Corporate L1',
       'policy_Corporate L2', 'policy_Corporate L3', 'policy_Personal L1',
       'policy_Personal L2', 'policy_Personal L3', 'policy_Special L1',
       'policy_Special L2', 'policy_Special L3', 'renew_offer_type_Offer1',
       'renew_offer_type_Offer2', 'renew_offer_type_Offer3',
       'renew_offer_type_Offer4', 'sales_channel_Agent',
       'sales_channel_Branch', 'sales_channel_Call Center',
       'sales_channel_Web', 'vehicle_class_Four-Door Car',
       'vehicle_class_Luxury Car', 'vehicle_class_Luxury SUV',
       'vehicle_class_SUV', 'vehicle_class_Sports Car',
       'vehicle_class_Two-Door Car', 'vehicle_type_A']]
y = final_df[["total_claim_amount"]]

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [ ]:
#Standardize the data (after the data split!).

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

scaler = MinMaxScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
y_train.hist()

In [ ]:
pt = PowerTransformer()

In [ ]:
pt.fit(y_train)
y_train = pt.transform(y_train)

In [ ]:
y_test = pt.transform(y_test)

### Model Training

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
model = lm.fit(X_train,y_train)

In [ ]:
model.coef_

In [ ]:
model.intercept_

### Predict total claim amount

In [ ]:
random_claim = pd.DataFrame(X_test).sample()

In [ ]:
random_claim

In [ ]:
model.predict(random_claim)

In [ ]:
pt.inverse_transform(model.predict(random_claim))

### Create predictions for test set

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# inverse transforming predictions

In [ ]:
pt.inverse_transform(y_pred)

In [ ]:
y_pred = pd.DataFrame(y_pred)

In [ ]:
y_test = pd.DataFrame(y_test)

In [ ]:
y_test = y_test.reset_index(drop=True)

In [ ]:
residuals_df = pd.concat([y_test,y_pred],axis=1)

In [ ]:
residuals_df.columns = ["y_test","y_pred"]

In [ ]:
residuals_df

In [ ]:
residuals_df["residuals"] = residuals_df["y_test"]- residuals_df["y_pred"]

In [ ]:
residuals_df

### Calculating metrics

In [ ]:
mean_error = residuals_df["residuals"].mean()

In [ ]:
mean_error

In [ ]:
from sklearn.metrics import mean_squared_error as mse , mean_absolute_error as mae

In [ ]:
mse(y_test,y_pred)

In [ ]:
mae(y_test,y_pred)

In [ ]:
rmse = mse(y_test,y_pred, squared = False)
rmse

In [ ]:
from sklearn.metrics import r2_score

### Computing R2_score for test_set

In [ ]:
round(r2_score(y_test, y_pred),2)

### Computing R2_score_for_train_set

In [ ]:
y_pred_train = pd.DataFrame(model.predict(X_train))

In [ ]:
r2_score(y_train,y_pred_train)

In [ ]:
feature_names = list(X_train.columns)

In [ ]:
coefficients = model.coef_

In [ ]:
coefficients = [i for i in coefficients[0]]

In [ ]:
fi_dict = {"name":feature_names,"coeff":coefficients}
fi_df = pd.DataFrame(fi_dict)
fi_df

In [ ]:
import seaborn as sns
fi_df_sorted = fi_df.sort_values(by = "coeff", ascending = False)
import matplotlib.pyplot as plt
plt.figure(figsize = (10,12))
sns.barplot(y = fi_df_sorted["name"], x = fi_df_sorted["coeff"])

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(20, 8))

sns.barplot(x = fi_df["name"],y =fi_df["coeff"])

In [ ]:
X_train["location_code_Suburban"].describe()